In [62]:
# Load packages
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [63]:
path = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/intermediate/call_reports_legacy'

# set path as working directory:
os.chdir(path)

In [64]:
# Load data
riad = pd.read_csv('RIAD_legacy.csv')
rcfd = pd.read_csv('RCFD_legacy.csv')
rcfa = pd.read_csv('RCFA.csv')

In [65]:
# Rename column RSSD9999 to Date:
riad.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcfd.rename(columns={'RSSD9999':'Date', 'RSSD9010': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

rcfa.rename(columns={'RSSD9999':'Date', 'RSSD9017': 'Financial Institution Name', 
                     'RSSD9001': 'IDRSSD'}, inplace=True)

In [66]:
riad['Date'] = pd.to_datetime(riad['Date'], format='%Y%m%d')
rcfd['Date'] = pd.to_datetime(riad['Date'], format='%Y%m%d')
rcfa['Date'] = pd.to_datetime(riad['Date'], format='%Y%m%d')

## Testing Deposits

In [120]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year==2010) & (rcfd['Date'].dt.quarter==4)][['Date', 'Financial Institution Name', 'RCFD2200']]

,Date,Financial Institution Name,RCFD2200
596997,2010-12-31,BANK OF AMER NA,1.038171e+09


In [122]:
# SILLICON VALLEY BANK:
rcfd[(rcfd['IDRSSD']==802866) & (rcfd['Date'].dt.year<2007) & 
     (rcfd['Date'].dt.year>2000)][['Date', 'Financial Institution Name', 'RCFD2200']]

,Date,Financial Institution Name,RCFD2200
1000519,2001-03-31,SILICON VALLEY BK,4065675.0
1000520,2001-06-30,SILICON VALLEY BK,3666568.0
1000521,2001-09-30,SILICON VALLEY BK,3477299.0
1000522,2001-12-31,SILICON VALLEY BK,3395268.0
1000523,2002-03-31,SILICON VALLEY BK,3195018.0
1000524,2002-06-30,SILICON VALLEY BK,3011471.0
1000525,2002-09-30,SILICON VALLEY BK,3109589.0
1000526,2002-12-31,SILICON VALLEY BK,3445638.0
1000527,2003-03-31,SILICON VALLEY BK,3257976.0
1000528,2003-06-30,SILICON VALLEY BK,3494809.0


## Testing Loans

In [111]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year==2010)][['Date', 'IDRSSD', 'RCFD1400']]

,Date,IDRSSD,RCFD1400
596994,2010-03-31,480228,741621407.0
596995,2010-06-30,480228,739369114.0
596996,2010-09-30,480228,724632971.0
596997,2010-12-31,480228,738173018.0


In [113]:
rcfd[(rcfd['IDRSSD']==802866) & (rcfd['Date'].dt.year>2010)][['Date', 'IDRSSD', 'RCFD1400']]

,Date,IDRSSD,RCFD1400
1000559,2011-03-31,802866,NaN
1000560,2011-06-30,802866,NaN
1000561,2011-09-30,802866,NaN
1000562,2011-12-31,802866,NaN
1000563,2012-03-31,802866,NaN
1000564,2012-06-30,802866,NaN
1000565,2012-09-30,802866,NaN
1000566,2012-12-31,802866,NaN
1000567,2013-03-31,802866,NaN
1000568,2013-06-30,802866,NaN


## Testing Assets

In [127]:
rcfd[(rcfd['IDRSSD']==802866) & (rcfd['Date'].dt.year>2000)][['Date', 'Financial Institution Name','RCFD2170']]

,Date,Financial Institution Name,RCFD2170
1000519,2001-03-31,SILICON VALLEY BK,4643642.0
1000520,2001-06-30,SILICON VALLEY BK,4252514.0
1000521,2001-09-30,SILICON VALLEY BK,4012853.0
1000522,2001-12-31,SILICON VALLEY BK,3950355.0
1000523,2002-03-31,SILICON VALLEY BK,3762130.0
...,...,...,...
1000596,2020-06-30,SILICON VALLEY BK,84489625.0
1000597,2020-09-30,SILICON VALLEY BK,95182370.0
1000598,2020-12-31,SILICON VALLEY BK,113839098.0
1000599,2021-03-31,SILICON VALLEY BK,140260913.0


In [128]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>2000)][['Date', 'Financial Institution Name','RCFD2170']]

,Date,Financial Institution Name,RCFD2170
596958,2001-03-31,BANK OF AMER NA,5.535090e+08
596959,2001-06-30,BANK OF AMER NA,5.638440e+08
596960,2001-09-30,BANK OF AMER NA,5.750200e+08
596961,2001-12-31,BANK OF AMER NA,5.516910e+08
596962,2002-03-31,BANK OF AMER NA,5.406100e+08
...,...,...,...
597035,2020-06-30,BANK OF AMER NA,2.161656e+09
597036,2020-09-30,BANK OF AMER NA,2.157008e+09
597037,2020-12-31,BANK OF AMER NA,2.258832e+09
597038,2021-03-31,BANK OF AMER NA,2.316773e+09


## Testing FedFunds Purchased

In [132]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year<2002)][['Date', 'Financial Institution Name', 'RCFD2800']]

,Date,Financial Institution Name,RCFD2800
596890,1984-03-31,NCNB NB OF NC,1311610.0
596891,1984-06-30,NCNB NB OF NC,1831116.0
596892,1984-09-30,NCNB NB OF NC,1535259.0
596893,1984-12-31,NCNB NB OF NC,1637810.0
596894,1985-03-31,NCNB NB OF NC,1685666.0
...,...,...,...
596957,2000-12-31,BANK OF AMER NA,35807000.0
596958,2001-03-31,BANK OF AMER NA,24928000.0
596959,2001-06-30,BANK OF AMER NA,33605000.0
596960,2001-09-30,BANK OF AMER NA,38905000.0


In [134]:
rcfd[(rcfd['IDRSSD']==480228) & (rcfd['Date'].dt.year>2001)][['Date', 'Financial Institution Name', 'RCFDB993', 'RCFDB995']]

,Date,Financial Institution Name,RCFDB993,RCFDB995
596962,2002-03-31,BANK OF AMER NA,NaN,19443000.0
596963,2002-06-30,BANK OF AMER NA,6229000.0,22407000.0
596964,2002-09-30,BANK OF AMER NA,5054000.0,23781000.0
596965,2002-12-31,BANK OF AMER NA,5941000.0,23627000.0
596966,2003-03-31,BANK OF AMER NA,5686000.0,23910000.0
...,...,...,...,...
597035,2020-06-30,BANK OF AMER NA,NaN,36450000.0
597036,2020-09-30,BANK OF AMER NA,NaN,36745000.0
597037,2020-12-31,BANK OF AMER NA,NaN,35406000.0
597038,2021-03-31,BANK OF AMER NA,NaN,23557000.0


In [146]:
rcfd[(rcfd['IDRSSD']==802866) ][['Date', 'Financial Institution Name', 'RCFDB993', 'RCFDB995']]

,Date,Financial Institution Name,RCFDB993,RCFDB995
1000451,1984-03-31,SILICON VALLEY BK,NaN,NaN
1000452,1984-06-30,SILICON VALLEY BK,NaN,NaN
1000453,1984-09-30,SILICON VALLEY BK,NaN,NaN
1000454,1984-12-31,SILICON VALLEY BK,NaN,NaN
1000455,1985-03-31,SILICON VALLEY BK,NaN,NaN
...,...,...,...,...
1000596,2020-06-30,SILICON VALLEY BK,NaN,0.0
1000597,2020-09-30,SILICON VALLEY BK,NaN,0.0
1000598,2020-12-31,SILICON VALLEY BK,NaN,0.0
1000599,2021-03-31,SILICON VALLEY BK,NaN,0.0


## Testing Loans Non-Accrual

In [143]:
rcfd[(rcfd['IDRSSD']==480228)][['Date', 'Financial Institution Name', 'RCFD1403']]

,Date,Financial Institution Name,RCFD1403
596890,1984-03-31,NCNB NB OF NC,89634.0
596891,1984-06-30,NCNB NB OF NC,77861.0
596892,1984-09-30,NCNB NB OF NC,81983.0
596893,1984-12-31,NCNB NB OF NC,69077.0
596894,1985-03-31,NCNB NB OF NC,76955.0
...,...,...,...
597035,2020-06-30,BANK OF AMER NA,4431000.0
597036,2020-09-30,BANK OF AMER NA,4516000.0
597037,2020-12-31,BANK OF AMER NA,5094000.0
597038,2021-03-31,BANK OF AMER NA,5355000.0


In [144]:
rcfd[(rcfd['IDRSSD']==802866)][['Date', 'Financial Institution Name', 'RCFD1403']]


,Date,Financial Institution Name,RCFD1403
1000451,1984-03-31,SILICON VALLEY BK,0.0
1000452,1984-06-30,SILICON VALLEY BK,0.0
1000453,1984-09-30,SILICON VALLEY BK,10.0
1000454,1984-12-31,SILICON VALLEY BK,432.0
1000455,1985-03-31,SILICON VALLEY BK,496.0
...,...,...,...
1000596,2020-06-30,SILICON VALLEY BK,94326.0
1000597,2020-09-30,SILICON VALLEY BK,105710.0
1000598,2020-12-31,SILICON VALLEY BK,104244.0
1000599,2021-03-31,SILICON VALLEY BK,90247.0


## Test Loans Past Due 90 Days

## Dean's Variables

In [67]:
# concatenate the two dataframes:
df = pd.merge(riad, rcfd, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')
df = pd.merge(df, rcfa, on=['Date', 'IDRSSD', 'Financial Institution Name'], how='outer')

In [68]:
# create a 'Year' column:
df['Year'] = df['Date'].dt.year

# create a 'Quarter' column:
df['Quarter'] = df['Date'].dt.quarter

In [69]:
df.rename(columns={ # RCFD variables (Balance Sheet):
                'RCFD2170':'Total Assets', 'RCFD1400': 'Loans', 'RCFD2200': 'Deposits', 
                'RCFD1403': 'Loans Non-Accrual', 'RCFD1407':'Loans Past Due 90 Days',
                'RCFD0010': 'Cash', 
                # RIAD variables (Income Statement):
                'RIAD4170':'Interest Exp. Deposits', 
                'RIAD4180': 'Interest Exp. FedFunds', 'RIAD4635': 'Charge Off Loans', 
                'RIAD4605': 'Recovery Loans', 'RIAD4130': 'Total Expenses', 
                'RIAD4217': 'Expenses on premises and fixed assets', 
                'RIAD4135': 'Labor expenses', 'RIAD4079': 'Total Non-interest Income'}, inplace=True)

In [ ]:
df['Interest Income Loans'] = df['RIAD4010'].fillna(0)+df['RIAD4065'].fillna(0)
df['Dividends'] = df['RIAD4470'].fillna(0)+df['RIAD4460'].fillna(0)

In [71]:
df['Federal Funds Purchased'] = np.where(df['Date'].dt.year < 2002, df['RCFD2800'], df['RCFDB993']+df['RCFDB995'])
df['Tier 1 Capital'] = np.where(df['Date'].dt.year < 2014, df['RCFD8274'], df['RCFA8274'])
df['Risk-Weighted Assets'] = np.where(df['Date'].dt.year < 2014, df['RCFDA223'], df['RCFAA223'])
df['Other Borrowings'] = np.where(df['Date'].dt.year < 2001, df['RCFD2835'], df['RCFD3190'])
df['FedFunds Sold'] = np.where(df['Date'].dt.year < 2002, df['RCFD1350'], df['RCFDB987']+df['RCFDB989'])
df['US Treasury Securities'] = np.where(df['Date'].dt.year < 1994, df['RCFD0400'], df['RCFD0211']+df['RCFD1287'])

In [ ]:
# For 'US Agency Obligations' we need a np.where with 3 time windows (1984-1993, 1994-2008, 2009-2010, 2011-2021):
df['US Agency Obligations'] = np.where(
                            df['Date'].dt.year < 1994, df['RCFD0600'],
                            np.where(
                            df['Date'].dt.year < 2009, 
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFD1698'].fillna(0)+df['RCFD1702'].fillna(0)+df['RCFD1703'].fillna(0)+df['RCFD1707'].fillna(0)+
                            df['RCFD1714'].fillna(0)+df['RCFD1717'].fillna(0)+df['RCFD1718'].fillna(0)+df['RCFD1732'].fillna(0),
                            np.where(
                            df['Date'].dt.year < 2011, 
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFDG300'].fillna(0)+df['RCFDG303'].fillna(0)+df['RCFDG304'].fillna(0)+df['RCFDG307'].fillna(0)+
                            df['RCFDG312'].fillna(0)+df['RCFDG315'].fillna(0)+df['RCFDG316'].fillna(0)+df['RCFDG319'].fillna(0)+
                            df['RCFDG324'].fillna(0)+df['RCFDG327'].fillna(0),
                            df['RCFD1289'].fillna(0)+df['RCFD1294'].fillna(0)+df['RCFD1293'].fillna(0)+df['RCFD1298'].fillna(0)+
                            df['RCFDG300'].fillna(0)+df['RCFDG303'].fillna(0)+df['RCFDG304'].fillna(0)+df['RCFDG307'].fillna(0)+
                            df['RCFDG312'].fillna(0)+df['RCFDG315'].fillna(0)+df['RCFDG316'].fillna(0)+df['RCFDG319'].fillna(0) 
# Missing here RCFDK142 and RCFDK145.
                            )
                            )
)

In [ ]:
df['Interest Income Safe Securities'] = np.where(df['Date'].dt.year<2001, df['RIAD4027'], df['RIADB488'])
df['Equity Issuance'] = np.where(df['Date'].dt.year<2001, df['RIAD4346'].fillna(0)+df['RIADB510'].fillna(0), 
                                 df['RIADB509'].fillna(0)+df['RIADB510'].fillna(0))

In [74]:
# Derived variables:
df['Loans (RW Assets) to Assets'] = df['Risk-Weighted Assets']/df['Total Assets']
df['Cash and Securities to Assets'] = 1 - df['Loans (RW Assets) to Assets']
df['Capital Ratio (risk-weighted)'] = df['Tier 1 Capital']/df['Risk-Weighted Assets']
df['Equity to Assets'] = df['Tier 1 Capital']/df['Total Assets']
df['Deposits to Assets'] = 1 - df['Equity to Assets']
df['Interest Return on Loans'] = df['Interest Income Loans']/df['Loans']
df['Interest Cost Deposits'] = df['Interest Exp. Deposits']/df['Deposits']
df['Loan Interest Margin'] = df['Interest Income Loans']-df['Interest Exp. Deposits']
df['Cost Fed Funds'] = df['Interest Exp. FedFunds']/df['Federal Funds Purchased']
df['Charge Off Rate Loans']  = (df['Charge Off Loans']-df['Recovery Loans'])/df['Loans']
df['Delinquency Rate Loans'] = (df['Loans Non-Accrual']+df['Loans Past Due 90 Days'])/df['Loans']
df['Safe Securities'] = df['US Treasury Securities']+df['US Agency Obligations']
df['Cost of Funds'] = (df['Interest Exp. Deposits']+df['Interest Exp. FedFunds'])/(df['Deposits']+df['Federal Funds Purchased'])
df['Interest Return on Safe Assets'] = df['Interest Income Safe Securities'] / df['Safe Securities']
df['Return Safe Securities'] = df['Interest Return on Safe Assets'] - df['Cost of Funds']
df['Return on Loans'] = df['Interest Return on Loans'] - df['Charge Off Rate Loans']

In [75]:
# keep only the variables we created/renamed:
df = df[['Date', 'Year', 'Quarter', 'Financial Institution Name', 'IDRSSD', 'Total Assets', 'Loans', 'Deposits', 
         'Loans Non-Accrual', 'Loans Past Due 90 Days', 'Cash', 'Interest Income Loans', 'Dividends', 
         'Interest Exp. Deposits', 'Interest Exp. FedFunds', 'Charge Off Loans', 'Recovery Loans', 
         'Total Expenses', 'Expenses on premises and fixed assets', 'Labor expenses', 'Total Non-interest Income', 
         'Federal Funds Purchased', 'Tier 1 Capital', 'Risk-Weighted Assets', 'Other Borrowings', 'FedFunds Sold', 
         'US Treasury Securities', 'US Agency Obligations', 'Interest Income Safe Securities', 'Equity Issuance', 
         'Loans (RW Assets) to Assets', 'Cash and Securities to Assets', 'Capital Ratio (risk-weighted)', 
         'Equity to Assets', 'Deposits to Assets', 'Interest Return on Loans', 'Interest Cost Deposits', 
         'Loan Interest Margin', 'Cost Fed Funds', 'Charge Off Rate Loans', 'Delinquency Rate Loans', 
         'Safe Securities', 'Cost of Funds', 'Interest Return on Safe Assets', 'Return Safe Securities', 
         'Return on Loans']]

## M&A Merges

In [77]:
df2 = df.copy()

In [78]:
transformations = pd.read_csv('C:/Users/angel/Documents/Economics/Research/Banking Project/data/clean/CSV_TRANSFORMATIONS.csv')

transformations['DT_TRANS'] = transformations['DT_TRANS'].astype(str)

transformations['Year'] = transformations['DT_TRANS'].str[:4]
transformations['Month'] = transformations['DT_TRANS'].str[4:6]
transformations['Day'] = transformations['DT_TRANS'].str[6:8]
transformations['Transaction Date'] = pd.to_datetime(transformations['Year'] + '-' + transformations['Month'] + '-' + transformations['Day'])

# drop the Month, Day, and Year columns:
transformations.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

# Rename #ID_RSSD_PREDECESSOR to IDRSSD Predecessor:
transformations.rename(columns={'#ID_RSSD_PREDECESSOR':'IDRSSD Predecessor',
                                'ID_RSSD_SUCCESSOR': 'IDRSSD Successor', 
                                'TRNSFM_CD': 'Transformation Code'}, inplace=True)

# make those columns integers:
transformations['IDRSSD Predecessor'] = transformations['IDRSSD Predecessor'].astype(int)
transformations['IDRSSD Successor'] = transformations['IDRSSD Successor'].astype(int)

transformations = transformations[['Transaction Date', 'IDRSSD Predecessor', 'IDRSSD Successor', 'Transformation Code']]


In [79]:
df_predecessor = pd.merge(df2, transformations, left_on='IDRSSD', right_on='IDRSSD Predecessor', how='left')
df_successor = pd.merge(df2, transformations, left_on='IDRSSD', right_on='IDRSSD Successor', how='left')